<a href="https://colab.research.google.com/github/jcjoshuac/data_science/blob/main/TreeHacks_2023_YouTube_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's import all the tools we need for our YouTube data science project.

In [22]:
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

from torch.utils.data import Dataset, DataLoader

Let's read our YouTube statistics data (which comes in the form of a csv file) as a pandas dataframe and take a brief look at our data.

In [23]:
youtube_stats_df = pd.read_csv('USvideos.csv') # USvideos.csv dataset taken from Kaggle
youtube_stats_df.head(n=10)

ParserError: ignored

Let's import our machine learning tools (sklearn).

In [ ]:
from sklearn import linear_model as lm

linear_model_m1 = lm.LinearRegression(fit_intercept=True)
linear_model_m2 = lm.LinearRegression(fit_intercept=True)

In [ ]:
def train_test_split(data):
    """Split the data into training and test data"""
    
    data_len = data.shape[0]
    shuffled_indices = np.random.permutation(data_len)
    train_indices = shuffled_indices[:int(0.8 * data_len)]
    test_indices = shuffled_indices[int(0.8 * data_len):]
    return data.iloc[train_indices], data.iloc[test_indices]
    
train, test = train_test_split(youtube_stats_df)

In [ ]:
# Separate our X feature matrix and our y prediction column
youtube_stats_df_train_x = train[['views', 'dislikes', 'comment_count']] # train.drop(columns=["likes"])
youtube_stats_df_train_y = train["likes"]
youtube_stats_df_test_x = test[['views', 'dislikes', 'comment_count']]
youtube_stats_df_test_y = test["likes"]
youtube_stats_df_train_x

In [ ]:
# Fit the model
linear_model_m1.fit(youtube_stats_df_train_x, youtube_stats_df_train_y)

# Compute the fitted and predicted values
y_fitted_m1 = linear_model_m1.predict(youtube_stats_df_train_x)
y_predicted_m1 = linear_model_m1.predict(youtube_stats_df_test_x)

In [ ]:
predicted_and_actual = {"predicted": y_predicted_m1, "actual": youtube_stats_df_test_y}
predicted_and_actual_df = pd.DataFrame(predicted_and_actual)
predicted_and_actual_df

In [ ]:
# Plot our predicted and actual like counts with a line of best fit
plt.scatter(x=youtube_stats_df_test_y, y=y_predicted_m1, alpha=0.5, s=5)
plt.title("Predicted like count against actual like count")
plt.xlabel("Actual like count")
plt.ylabel("Predicted like count");

theta = np.polyfit(youtube_stats_df_test_y, y_predicted_m1, 1)
y_line = theta[1] + theta[0] * youtube_stats_df_test_y
plt.plot(youtube_stats_df_test_y, y_line, 'r')

print(f'Regression coefficient: {theta[0]}')
print("Percentage accuracy: ", str(theta[0] * 100) +"%")

Let's test our model on my (@jcjoshuac) Week in the Life of a UC Berkeley student YouTube video.

In [ ]:
week_in_the_life_features_df = (pd.DataFrame({"views": [8826], "dislikes": [5], "comment_count": [20]}))#.reset_index()
week_in_the_life_y_predicted = linear_model_m1.predict(week_in_the_life_features_df)
print("Predicted number of likes: " + str(week_in_the_life_y_predicted[0]))

Even though the model is highly accurate, the model performs poorly on this single data point.

Let's take a look at another YouTube statistics dataset.

In [ ]:
most_subscribed_channels = pd.read_csv('most_subscribed_youtube_channels.csv') # most_subscribed_youtube_channels.csv dataset taken from Kaggle
most_subscribed_channels.head(n=10)